# Measuring Cell Deformation

## Installs, imports, and uploads

In [1]:
# #@title Package Installations

# !pip install numpy opencv-python-headless pandas matplotlib stardist ipywidgets plotly --quiet



In [2]:
%matplotlib widget

In [3]:
# #@title File Upload

# from google.colab import files
# uploaded = files.upload()

# Just in case we're using this notebook from Colab

In [15]:
#@title Imports

import copy

import cv2
import numpy as np
from stardist.models import StarDist2D
from skimage import io, color
import pandas as pd

import matplotlib.pyplot as plt

from scipy.spatial.distance import cdist

from ipywidgets import Button, HBox, interact, IntSlider, Output
from IPython.display import display, clear_output
import ipywidgets as widgets

from skimage.morphology import opening, disk, closing
from skimage.filters import gaussian
from skimage.color import label2rgb

import pandas as pd
from skimage.measure import regionprops_table





## Loading Video and Frames

In [5]:
#@title Load Model and Data

# Load the pre-trained StarDist model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

# Initialize a list to store segmentation results
segmentation_results = []

# Load the video
video_path = 'Movie 11 - Fig. S31_actual 32.avi'
cap = cv2.VideoCapture(video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Preload frames (we could load a subset if the video was very large)
frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame_rgb[:, :, 1])  # Store only the green channel

cap.release()

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [6]:
#@title Rendering the Original Video

current_frame_idx = 0  # Track the current frame index

plot_output = widgets.Output()  # Dedicated output widget for the plot

def plot_frame(frame_idx):
    with plot_output:
        clear_output(wait=True)  # Clear the previous plot
        plt.figure(figsize=(8, 8))
        plt.imshow(frames[frame_idx], cmap='Greens')
        plt.title(f"Frame {frame_idx}")
        plt.axis('off')
        plt.show()

def on_previous_clicked(b):
    global current_frame_idx
    if current_frame_idx > 0:
        current_frame_idx -= 1
        slider.value = current_frame_idx  # This triggers plot update

def on_next_clicked(b):
    global current_frame_idx
    if current_frame_idx < len(frames) - 1:
        current_frame_idx += 1
        slider.value = current_frame_idx  # This triggers plot update

previous_button = Button(description="Previous")
next_button = Button(description="Next")

previous_button.on_click(on_previous_clicked)
next_button.on_click(on_next_clicked)

slider = IntSlider(min=0, max=len(frames)-1, step=1, value=0)

def on_slider_change(change):
    global current_frame_idx
    current_frame_idx = change['new']
    plot_frame(current_frame_idx)  # Update plot directly for smoother slider action

slider.observe(on_slider_change, names='value')

controls = HBox([previous_button, next_button, slider])  # Group the controls

# Display the controls and the initial plot separately
display(controls)
display(plot_output)  # This will hold the plot
plot_frame(current_frame_idx)  # Initial plot


Output()

## Preprocessing (denoising)

In [7]:

# Apply morphological opening to each frame to remove noise
# Disk-shaped structuring element with a radius of 3 pixels
selem = disk(3)

# We will store the processed frames in a separate list
processed_frames = copy.deepcopy(frames)

# Apply Gaussian blur to each opened frame (sigma controls the amount of blurring)
processed_frames = [gaussian(frame, sigma=2) for frame in processed_frames]

# Apply morphological opening to each frame to remove noise
processed_frames = [opening(frame, selem) for frame in processed_frames]

# processed_frames = [opening(frame, selem) for frame in processed_frames]
# processed_frames = [closing(frame, selem) for frame in processed_frames]
# processed_frames = [opening(frame, selem) for frame in processed_frames]
# processed_frames = [closing(frame, selem) for frame in processed_frames]


In [8]:
#@title Rendering the Processed Video

current_frame_idx = 0  # Track the current frame index

plot_output = widgets.Output()  # Dedicated output widget for the plot

def plot_frame(frame_idx):
    with plot_output:
        clear_output(wait=True)  # Clear the previous plot
        plt.figure(figsize=(8, 8))
        plt.imshow(processed_frames[frame_idx], cmap='Greens')
        plt.title(f"Frame {frame_idx}")
        plt.axis('off')
        plt.show()

def on_previous_clicked(b):
    global current_frame_idx
    if current_frame_idx > 0:
        current_frame_idx -= 1
        slider.value = current_frame_idx  # This triggers plot update

def on_next_clicked(b):
    global current_frame_idx
    if current_frame_idx < len(processed_frames) - 1:
        current_frame_idx += 1
        slider.value = current_frame_idx  # This triggers plot update

previous_button = Button(description="Previous")
next_button = Button(description="Next")

previous_button.on_click(on_previous_clicked)
next_button.on_click(on_next_clicked)

slider = IntSlider(min=0, max=len(processed_frames)-1, step=1, value=0)

def on_slider_change(change):
    global current_frame_idx
    current_frame_idx = change['new']
    plot_frame(current_frame_idx)  # Update plot directly for smoother slider action

slider.observe(on_slider_change, names='value')

controls = HBox([previous_button, next_button, slider])  # Group the controls

# Display the controls and the initial plot separately
display(controls)
display(plot_output)  # This will hold the plot
plot_frame(current_frame_idx)  # Initial plot


Output()

## Instance Segmentation

In [9]:

# Perform segmentation on each opened frame
segmentation_labels = [model.predict_instances(frame)[0] for frame in processed_frames]


In [10]:
#@title Rendering the Segmented Video

segmentation_output = Output()  # Dedicated output widget for segmentation plots

def plot_segmentation(frame_idx):
    with segmentation_output:
        clear_output(wait=True)
        label_overlay = label2rgb(segmentation_labels[frame_idx], image=processed_frames[frame_idx], bg_label=0, alpha=0.4)
        
        plt.figure(figsize=(8, 8))
        plt.imshow(label_overlay)
        plt.title(f"Segmented Frame {frame_idx} (with Gaussian Blur)")
        plt.axis('off')
        plt.show()

def on_previous_clicked(b):
    global current_frame_idx
    if current_frame_idx > 0:
        current_frame_idx -= 1
        slider.value = current_frame_idx  # This triggers plot update

def on_next_clicked(b):
    global current_frame_idx
    if current_frame_idx < len(processed_frames) - 1:
        current_frame_idx += 1
        slider.value = current_frame_idx  # This triggers plot update

previous_button = Button(description="Previous")
next_button = Button(description="Next")

previous_button.on_click(on_previous_clicked)
next_button.on_click(on_next_clicked)

slider = IntSlider(min=0, max=len(processed_frames)-1, step=1, value=0)

def on_slider_change(change):
    global current_frame_idx
    current_frame_idx = change['new']
    plot_segmentation(current_frame_idx)  # Directly update the plot for segmentation results

slider.observe(on_slider_change, names='value')

controls = HBox([previous_button, next_button, slider])

# Display the controls and the initial segmentation plot separately
display(controls)
display(segmentation_output)  # This will hold the segmentation plot
plot_segmentation(current_frame_idx)  # Initial plot


Output()

In [22]:
from skimage.measure import regionprops
import matplotlib.pyplot as plt
from skimage.color import label2rgb
from ipywidgets import Button, HBox, IntSlider, Output
from IPython.display import display, clear_output
import ipywidgets as widgets

segmentation_output = Output()

def plot_segmentation(frame_idx):
    with segmentation_output:
        clear_output(wait=True)
        label_overlay = label2rgb(segmentation_labels[frame_idx], image=processed_frames[frame_idx], bg_label=0, alpha=0.4)
        
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(label_overlay)
        
        # Iterate over all region properties
        for region in regionprops(segmentation_labels[frame_idx]):
            # Get the coordinates of the centroid of the current region
            y0, x0 = region.centroid
            # Use the label as the annotation text
            ax.text(x0, y0, str(region.label), color='cyan', fontsize=12, ha='center', va='center')
        
        ax.set_title(f"Segmented Frame {frame_idx} (with Gaussian Blur)")
        ax.axis('off')
        plt.show()

def on_previous_clicked(b):
    global current_frame_idx
    if current_frame_idx > 0:
        current_frame_idx -= 1
        slider.value = current_frame_idx  # This triggers plot update

def on_next_clicked(b):
    global current_frame_idx
    if current_frame_idx < len(processed_frames) - 1:
        current_frame_idx += 1
        slider.value = current_frame_idx  # This triggers plot update

previous_button = Button(description="Previous")
next_button = Button(description="Next")

previous_button.on_click(on_previous_clicked)
next_button.on_click(on_next_clicked)

slider = IntSlider(min=0, max=len(processed_frames)-1, step=1, value=0)

def on_slider_change(change):
    global current_frame_idx
    current_frame_idx = change['new']
    plot_segmentation(current_frame_idx)  # Directly update the plot for segmentation results

slider.observe(on_slider_change, names='value')

controls = HBox([previous_button, next_button, slider])

# Display the controls and the initial segmentation plot separately
display(controls)
display(segmentation_output)
plot_segmentation(current_frame_idx)  # Initial plot


Output()

## Extracting Properties

In [16]:
properties = [
    "label", "area", "centroid", "equivalent_diameter", "bbox_area", "convex_area", "filled_area", "major_axis_length",
    "minor_axis_length", "bbox", "weighted_centroid", "coords",
    "eccentricity", "euler_number", "extent",
    "feret_diameter_max", "max_intensity", "mean_intensity", "min_intensity",
    "orientation", "perimeter", "solidity"
]

# Keep in mind that `segmentation_labels` is a list of label matrices for
# each frame, and `processed_frames` is a list of processed frames
# corresponding to `segmentation_labels`

# Initialize an empty DataFrame to store results
properties_df = pd.DataFrame()

for frame_idx, labels in enumerate(segmentation_labels):
    # Extract properties for current frame using regionprops_table
    frame_props = regionprops_table(labels, intensity_image=processed_frames[frame_idx], properties=properties)
    
    # Convert the properties dictionary to a DataFrame
    frame_props_df = pd.DataFrame(frame_props)
    
    # Add a column for the current frame index
    frame_props_df['frame'] = frame_idx
    
    # Append the current frame's properties to the main DataFrame
    properties_df = pd.concat([properties_df, frame_props_df], ignore_index=True)

# Optional: Convert 'centroid-0' and 'centroid-1' to more descriptive column names (if required)
# properties_df.rename(columns={'centroid-0': 'centroid_y', 'centroid-1': 'centroid_x'}, inplace=True)

# Ensure "frame" is the first column
columns = ['frame'] + [col for col in properties_df.columns if col != 'frame']
properties_df = properties_df.reindex(columns=columns)

In [20]:
properties_df

,frame,label,area,centroid-0,centroid-1,equivalent_diameter,bbox_area,convex_area,filled_area,major_axis_length,...,eccentricity,euler_number,extent,feret_diameter_max,max_intensity,mean_intensity,min_intensity,orientation,perimeter,solidity
0,0,1,426.0,272.265258,314.678404,23.289484,572.0,443.0,426.0,25.540585,...,0.553044,1,0.744755,26.683328,0.986102,0.785875,0.365068,-1.521081,75.597980,0.961625
1,0,2,522.0,479.187739,420.034483,25.780439,768.0,543.0,522.0,34.126061,...,0.805799,1,0.679688,34.539832,0.577496,0.426793,0.216847,-0.402137,89.254834,0.961326
2,0,3,162.0,287.722222,158.283951,14.361922,210.0,167.0,162.0,15.867860,...,0.573088,1,0.771429,16.552945,0.769599,0.581683,0.276599,0.573992,44.627417,0.970060
3,0,4,274.0,343.609489,101.244526,18.677999,374.0,289.0,274.0,21.462682,...,0.644358,1,0.732620,22.803509,0.587235,0.409188,0.226100,1.248829,61.698485,0.948097
4,0,5,320.0,216.196875,488.996875,20.185060,414.0,331.0,320.0,23.315205,...,0.657523,1,0.772947,24.041631,0.368080,0.316646,0.167153,1.464430,65.112698,0.966767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59651,219,254,113.0,76.672566,234.707965,11.994835,154.0,128.0,113.0,14.473400,...,0.696960,1,0.733766,14.866069,0.204821,0.182541,0.150004,-1.559934,40.384776,0.882812
59652,219,255,308.0,665.487013,650.087662,19.802974,525.0,344.0,308.0,36.536051,...,0.950989,1,0.586667,36.138622,0.374785,0.235689,0.160192,-1.294954,81.941125,0.895349
59653,219,256,877.0,273.378563,72.452680,33.416030,1190.0,924.0,877.0,40.667307,...,0.730844,1,0.736975,41.617304,0.229240,0.186521,0.079539,-0.803835,114.083261,0.949134
59654,219,257,167.0,140.011976,14.964072,14.581872,225.0,175.0,167.0,15.558155,...,0.443601,1,0.742222,16.552945,0.263896,0.209832,0.163281,0.817410,46.627417,0.954286


## Object Tracking
Since segmentation is not very good (some objects are segmented as many), my approach will be to manually provide a dictionary with frames as keys (integers), and lists of labels (objects, so lists of integers) as values. Then, we'll filter the original dataframe to keep only the selected objects on each frame. With this new dataframe, we will be able to start the analysis.

In [30]:
# When labels overlap, use this cell to query if the label you're seeing
# is the correct one. Remember that you can see the coordinates of your mouse
# by right clicking in jupyter lab, and then selecting “create new view for
# cell ouput”. Use that coordinates to compare with centroid coordinates.

properties_df.loc[(properties_df['frame'] == 4) & (properties_df['label'].isin([230, 83]))][['frame', 'label', 'centroid-0', 'centroid-1']]

# Note: in the “view for cell output”, there's also a “zoom to rectangle” tool

,frame,label,centroid-0,centroid-1
1231,4,83,848.686047,356.104651
1378,4,230,848.689189,344.351351


In [ ]:
# Example object_to_track dictionary
object_to_track = {
    0: [10, 178, 230, 47],
    1: [4, 156, 170, 238],
    2: [5, 133],
    3: [6, 146, 48],
    4: [26, 182, 230, 83],
    5: [55, 104, 238, 146, 254],
}

# Initialize an empty DataFrame to store filtered results
filtered_df = pd.DataFrame()

for frame, labels in object_to_track.items():
    # Filter for the current frame and specified labels
    frame_filtered = df_properties[(df_properties['frame'] == frame) & (df_properties['label'].isin(labels))]
    
    # Append the filtered DataFrame to the cumulative DataFrame
    filtered_df = pd.concat([filtered_df, frame_filtered], ignore_index=True)

# Now filtered_df contains only the rows for specified objects in each frame


In [ ]:
# Let's check that the centroids of all objects in the same
# frame of filtered_df are closer than certain threshold.
# This assert is just to avoid stupid errors.

# ...

## Data Analysis
We'll start by making a new dataframe with a row for each frame and a coords column that in the union of all coordinates in the objects of that frame, effectively merging the selected objects into one. Then, we should calculate:
- the area by frame;
- PCA (we could use the variance or min/max values over these directions to get the aspect ratio of the objects).